In [1]:
import sys
sys.path.append('/scratch/rpradata/PressureEv/Simulations')

from Turbcopy.p3d import p3d

import numpy as np

import matplotlib.pyplot as plt

from numpy import newaxis as nna

from scipy.stats import norm

import matplotlib.mlab as mlab

import h5py

import pandas as pd

In [ ]:
# N = 120  # number of timesteps (change if needed)
# ny, nx = 4096, 4096

# with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/pressure-scalar-ion/pressure-scalar-ion.h5", "w") as f: ## EDIT
#     f.create_dataset("conv", (N, ny, nx), dtype='float32',
#                      chunks=(1, 256, 256), compression="gzip", compression_opts=4)
#     f.create_dataset("BWP",  (N, ny, nx), dtype='float32',
#                      chunks=(1, 256, 256), compression="gzip", compression_opts=4)
#     f.create_dataset("DP",   (N, ny, nx), dtype='float32',
#                      chunks=(1, 256, 256), compression="gzip", compression_opts=4)
#     f.create_dataset("comp", (N, ny, nx), dtype='float32',
#                      chunks=(1, 256, 256), compression="gzip", compression_opts=4)
#     f.create_dataset("dPdt", (N, ny, nx), dtype='float32',
#                      chunks=(1, 256, 256), compression="gzip", compression_opts=4)




In [11]:
with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/fixed_tensors_xxyyzzxy/fix_rhlh_terms_ion_xx.h5", "r") as f: #EDIT AS NAMED
    # read full array at timestep 0
    # conv0 = f["conv"][0, :, :]

    #with all timesteps:
    convxx = f["conv"][119,:,:]



with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/dpdt_ion/dpdt_ion_xx.h5", "r") as f: #EDIT AS NAMED

    dpdtxx = f["dpdt"][118,:,:] ##data only until t = 118 instead of t = 119. the remaining one is just empty.

In [17]:
def compute_scalar_p_ion(tdown, tup, save_dir):

    for ts_idx, ts in enumerate(range(tdown, tup)):

        with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/fixed_tensors_xxyyzzxy/fix_rhlh_terms_ion_xx.h5", "r") as f: #EDIT AS NAMED
            # read full array at timestep 0
            # conv0 = f["conv"][0, :, :]

            #with all timesteps:
            convxx = f["conv"][ts,:,:]
            BWPxx = f['BWP'][ts,:,:]
            DPxx = f['DP'][ts,:,:]
            compxx = f['comp'][ts,:,:]

        with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/fixed_tensors_xxyyzzxy/fix_rhlh_terms_ion_yy.h5", "r") as f: #EDIT AS NAMED
            # read full array at timestep 0
            # conv0 = f["conv"][0, :, :]

            #with all timesteps:
            convyy = f["conv"][ts,:,:]
            BWPyy = f['BWP'][ts,:,:]
            DPyy = f['DP'][ts,:,:]
            compyy = f['comp'][ts,:,:]

        with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/fixed_tensors_xxyyzzxy/fix_rhlh_terms_ion_zz.h5", "r") as f: #EDIT AS NAMED
            # read full array at timestep 0
            # conv0 = f["conv"][0, :, :]

            #with all timesteps:
            convzz = f["conv"][ts,:,:]
            BWPzz = f['BWP'][ts,:,:]
            DPzz = f['DP'][ts,:,:]
            compzz = f['comp'][ts,:,:]

        conv_tot = (1/3)*(convxx+convyy+convzz)
        BWP_tot = (1/3)*(BWPxx+BWPyy+BWPzz)
        DP_tot = (1/3)*(DPxx + DPyy + DPzz)
        comp_tot = (1/3)*(compxx + compyy + compzz)

        # print('check 1')

        with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/dpdt_ion/dpdt_ion_xx.h5", "r") as f: #EDIT AS NAMED
            dpdtxx = f["dpdt"][ts-1,:,:]

        with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/dpdt_ion/dpdt_ion_yy.h5", "r") as f: #EDIT AS NAMED
            dpdtyy = f["dpdt"][ts-1,:,:]

        with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/dpdt_ion/dpdt_ion_zz.h5", "r") as f: #EDIT AS NAMED
            dpdtzz = f["dpdt"][ts-1,:,:]

        dpdt_tot = (1/3)*(dpdtxx + dpdtyy + dpdtzz)

        # print('check 2')

        with h5py.File("%s/pressure-scalar-ion.h5" % (save_dir), "a") as f:  # "a" = append/update mode ##EDIT
            f["BWP"][ts, :, :]  = BWP_tot.astype('float32')
            f["DP"][ts, :, :]   = DP_tot.astype('float32')
            f["comp"][ts, :, :] = comp_tot.astype('float32')
            f["conv"][ts, :, :] = conv_tot.astype('float32')
            f["dPdt"][ts, :, :] = dpdt_tot.astype('float32')

        print(f"Saved ts={ts}")







In [ ]:
compute_scalar_p_ion(1, 119, '/scratch/rpradata/PressureEv/Simulations/ion-terms/pressure-scalar-ion')

In [2]:
#compute time avg

with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/pressure-scalar-ion/pressure-scalar-ion.h5", "r") as f: #EDIT AS NAMED
    # read full array at timestep 0
    # conv0 = f["conv"][0, :, :]

    #with all timesteps:
    conv = f["conv"]
    BWP = f['BWP']
    DP = f['DP']
    comp = f['comp']
    dpdt = f['dPdt']

    # read a smaller subregion without loading the entire file
    # subregion = f["BWP"][10, 0:512, 0:512]

    # check shape
    # print(conv0.shape, convt.shape, len(convt))

    conv_out = np.zeros(conv.shape[1:], dtype=np.float64)
    BWP_out = np.zeros(conv.shape[1:], dtype=np.float64)
    DP_out = np.zeros(conv.shape[1:], dtype=np.float64)
    comp_out = np.zeros(conv.shape[1:], dtype=np.float64)
    dpdt_out = np.zeros(dpdt.shape[1:], dtype=np.float64)

    for t in range(conv.shape[0]):     # loop over time axis
        conv_out += conv[t, :, :]          # adds one 4096x4096 slice at a time
        BWP_out += BWP[t, :, :]
        DP_out += DP[t, :, :]
        comp_out += comp[t, :, :]
        dpdt_out += dpdt[t, :, :]
     

# out now holds the time-sum, shape (4096, 4096)


###

conv_avg = conv_out / int(conv.shape[0])
BWP_avg = BWP_out / int(conv.shape[0])
DP_avg = DP_out / int(conv.shape[0])
comp_avg = comp_out / int(conv.shape[0])

dpdt_avg = dpdt_out/int(dpdt.shape[0])

##

k = 1  # number of timesteps (change if needed)
ny, nx = 4096, 4096

with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/pressure-scalar-ion/sumavg_ps_ion.h5", "w") as f: #EDIT THIS
    f.create_dataset("conv", (k, ny, nx), dtype='float32',
                     chunks=(1, 256, 256), compression="gzip", compression_opts=4)
    f.create_dataset("BWP",  (k, ny, nx), dtype='float32',
                     chunks=(1, 256, 256), compression="gzip", compression_opts=4)
    f.create_dataset("DP",   (k, ny, nx), dtype='float32',
                     chunks=(1, 256, 256), compression="gzip", compression_opts=4)
    f.create_dataset("comp", (k, ny, nx), dtype='float32',
                     chunks=(1, 256, 256), compression="gzip", compression_opts=4)
    f.create_dataset("dPdt", (k, ny, nx), dtype='float32',
                     chunks=(1, 256, 256), compression="gzip", compression_opts=4)

with h5py.File("/scratch/rpradata/PressureEv/Simulations/ion-terms/pressure-scalar-ion/sumavg_ps_ion.h5", "a") as f:  # "a" = append/update mode #EDIT THIS TOO
    f["conv"][0, :, :] = conv_avg.astype(np.float32)
    f["BWP"][0, :, :]  = BWP_avg.astype(np.float32)
    f["DP"][0, :, :]   = DP_avg.astype(np.float32)
    f["comp"][0, :, :] = comp_avg.astype(np.float32)
    f["dPdt"][0, :, :] = dpdt_avg.astype(np.float32)
